In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import re

### Actors: Importing the necessary data

I used the following three data sets to construct the functions computing the score:

In [3]:
actors_awards=pd.read_pickle('data/complete_stars_awards.pkl')

In [124]:
cast_movies=pd.read_pickle('data/complete_cast.pkl')
movies=pd.read_pickle('data/filtered_movies.pkl')

In [25]:
cast_movies.head()

,tconst,cast,count
0,tt0788137,"[nm2213445, nm2216523, nm2212320, nm2219080, n...",6
1,tt0788133,"[nm2100867, nm1952207, nm2218383, nm5041940, n...",6
2,tt0788141,"[nm0149867, nm0708218, nm0899804]",3
3,tt0788134,"[nm1833642, nm1044987, nm2756644, nm1833094, n...",5
4,tt0790463,"[nm2214417, nm2213293]",2


In [26]:
movies.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
145,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,20,"Documentary,News,Sport"
332,tt0000335,movie,Soldiers of the Cross,Soldiers of the Cross,0,1900,\N,\N,"Biography,Drama"
499,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N
571,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Biography,Crime,Drama"


In [63]:
all_movies=movies.merge(cast_movies,left_on='tconst',right_on='tconst')

In [30]:
all_movies.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,cast,count
0,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance,"[nm0063086, nm0183823, nm1309758]",3
1,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,20,"Documentary,News,Sport","[nm0179163, nm0280615, nm4082222, nm4081458, n...",5
2,tt0000335,movie,Soldiers of the Cross,Soldiers of the Cross,0,1900,\N,\N,"Biography,Drama","[nm1010955, nm1012612, nm1011210, nm1012621, n...",6
3,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N,"[nm0215752, nm0252720]",2
4,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Biography,Crime,Drama","[nm0846887, nm0846894, nm3002376, nm0170118, n...",15


In [4]:
actors_awards.head()

,nconst,year,category,w_n,description,movie,tconst
0,nm0076183,2001,Silver Condor,Nominee,Best First Film (Mejor Ópera Prima),76-89-03,tt0247905
1,nm0076183,2007,Best Latin-American Film,Nominee,Latin-American Competition,Regresados,tt0362080
2,nm0076183,2001,MTV Movie Award,Nominee,MTV South Feed (mostly Argentina) - Favorite F...,76-89-03,tt0247905
3,nm0248444,2007,Silver Condor,Nominee,Best Video Film (Mejor Videofilm),Pacto de silencio,tt0476327
4,nm0248444,2019,Human Rights Competition,Nominee,None,"Chubut, libertad y tierra",tt10670592


In [6]:
actors_awards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382947 entries, 0 to 382946
Data columns (total 7 columns):
nconst         382947 non-null object
year           382947 non-null int64
category       382947 non-null object
w_n            382947 non-null object
description    338149 non-null object
movie          344264 non-null object
tconst         344265 non-null object
dtypes: int64(1), object(6)
memory usage: 20.5+ MB


In [4]:
#"Palme d'Or" , we will remove the caractere " ' " to do the filtering
actors_awards['category']=actors_awards.apply(lambda x:  x['category'].replace("'",' ') ,axis=1)

In [47]:
actors_awards[actors_awards.apply(lambda x: 'Palme' in x.category ,axis=1)]

,nconst,year,category,w_n,description,movie,tconst
124,nm0140241,1964,Golden Berlin Bear,Nominee,None,Los evadidos,tt0183058
683,nm0000180,1954,Golden Berlin Bear,Winner,None,Hobson's Choice,tt0047094
1170,nm0127287,1978,Golden Berlin Bear,Nominee,None,Flammende Herzen,tt0077560
3205,nm0801446,1978,Golden Berlin Bear,Nominee,None,Apám néhány boldog éve,tt0075693
4100,nm0001226,1986,Golden Berlin Bear,Nominee,None,At Close Range,tt0090670
...,...,...,...,...,...,...,...
381050,nm0906667,1980,Golden Berlin Bear,Nominee,None,Dyrygent,tt0079090
381051,nm0906667,1968,Golden Berlin Bear,Nominee,None,Gates to Paradise,tt0062461
381052,nm0906667,1962,Golden Berlin Bear,Nominee,None,L'amour à vingt ans,tt0055747
381631,nm0136021,1987,Golden Berlin Bear,Nominee,None,Mauvais sang,tt0091497


#### Filtering awards

Main actors awards:

In [5]:
Awards=['Oscar','Palme d Or','BAFTA Film Award','Golden Berlin Bear','Filmfare Award','European Film Award','Golden Leopard','Grand Jury Prize','Golden Globe','Golden Lion']

filtering:

In [6]:
actors_awards=actors_awards[actors_awards.apply(lambda x: x.category in Awards , axis=1)]

In [53]:
actors_awards.head(10)

,nconst,year,category,w_n,description,movie,tconst
66,nm0070148,1959,Palme d Or,Nominee,None,Araya,tt0051372
120,nm0079244,1985,Golden Lion,Nominee,None,Réquiem por un campesino español,tt0089946
124,nm0140241,1964,Golden Berlin Bear,Nominee,None,Los evadidos,tt0183058
179,nm2150974,2016,Golden Globe,Winner,Best Actor,Amor Impossível,tt4603080
194,nm5650535,2019,Grand Jury Prize,Winner,Graveyard Shift Competition,Climate of the Hunter,tt9755252
358,nm4392757,2019,Grand Jury Prize,Winner,Best Experimental Short,Endless Summer for the Queer Mind,tt8959646
367,nm1061987,1988,Golden Globe,Nominee,Best Original Song - Motion Picture,The Secret of My Success,tt0093936
373,nm0853050,2006,Oscar,Winner,Best Achievement in Visual Effects,King Kong,tt0360717
374,nm0853050,2004,Oscar,Winner,Best Costume Design,The Lord of the Rings: The Return of the King,tt0167260
375,nm0853050,2004,Oscar,Winner,Best Makeup,The Lord of the Rings: The Return of the King,tt0167260


# Actor score :

In [31]:
#This function computes score based on a weighted sum depending on the column (Winner\Nominee) and the cateory of the award.
def actor_score(categories,w_n):
    if(len(categories)==0):
        return 0
    else:
        #Giving weights to awards
        awards_weights=dict({'Oscar':10 ,'Golden Globe':9,'Palme d Or':8,'BAFTA Film Award':7,'Golden Berlin Bear':6,'Filmfare Award':5,'European Film Award':4,'Golden Leopard':3,'Grand Jury Prize':2,'Golden Lion':1})
        winner=dict({'Nominee':1,'Winner':2})
        score=0
        for category,win in zip(categories,w_n):
            score=score+winner[win]*awards_weights[category]
        return score    
       

In [32]:
#This function takes as imput the dataframe containing the awards of each actor, the name of a specific actor and a year
#It return the actor score of the given nconst before that year
def actor_year_score(df,nconst,year):
    n_df=df[(df['nconst']==nconst) & (df['year']<year)]
    return actor_score(n_df['category'],n_df['w_n'])
    

In [20]:
#Testing the actor_year_score for a given nconst
actors_awards[actors_awards.nconst=='nm0853050'].head(50)

,nconst,year,category,w_n,description,movie,tconst
373,nm0853050,2006,Oscar,Winner,Best Achievement in Visual Effects,King Kong,tt0360717
374,nm0853050,2004,Oscar,Winner,Best Costume Design,The Lord of the Rings: The Return of the King,tt0167260
375,nm0853050,2004,Oscar,Winner,Best Makeup,The Lord of the Rings: The Return of the King,tt0167260
376,nm0853050,2002,Oscar,Winner,Best Makeup,The Lord of the Rings: The Fellowship of the Ring,tt0120737
377,nm0853050,2002,Oscar,Winner,"Best Effects, Visual Effects",The Lord of the Rings: The Fellowship of the Ring,tt0120737
378,nm0853050,2002,Oscar,Nominee,Best Costume Design,The Lord of the Rings: The Fellowship of the Ring,tt0120737
379,nm0853050,2014,BAFTA Film Award,Nominee,Best Make Up/Hair,The Hobbit: The Desolation of Smaug,tt1170358
380,nm0853050,2013,BAFTA Film Award,Nominee,Best Make Up/Hair,The Hobbit: An Unexpected Journey,tt0903624
381,nm0853050,2006,BAFTA Film Award,Winner,Best Achievement in Special Visual Effects,King Kong,tt0360717
382,nm0853050,2004,BAFTA Film Award,Nominee,Best Costume Design,The Lord of the Rings: The Return of the King,tt0167260


In [33]:
actor_year_score(actors_awards,'nm0853050',2005)

153

In [70]:
#Cumulated score takes as an input the dataframe containing actors awards, the hole cast of a movie and a year
# and outputs the cumulated score : the sum of actor's scores present in the cast list:
def cumulated_score(df_actors,cast,year):
    #if the cast="[]"
    if len(cast)==0 :
        return 0
    #if the startYear of the movie is missing the cumulated score will return 0
    if (str(year)=='\\N'):
        return 0
    else:
        total_score=0
        for actor in cast:
            total_score=total_score+actor_year_score(df_actors,actor,year)
        return total_score    

In [97]:
#testing on titanic:
titanic=all_movies[all_movies.tconst=='tt0120338']
titanic

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,cast,count
82826,tt0120338,movie,Titanic,Titanic,0,1997,\N,194,"Drama,Romance","[nm0000138, nm0000701, nm0000708, nm0000870, n...",164


In [122]:
score=cumulated_score(actors_awards,titanic.iloc[0,9],int(titanic.iloc[0,5]))

In [123]:
score

113

In [58]:
#ignoring the missing years:
def no_null(year):
    if(year=="\\N"):
        return 0
    else:
        return int(year)
         

In [126]:
#testing on a subset of movies starting after 2000:
slice_movie=all_movies[all_movies.apply(lambda x: no_null(x.startYear)>2000 ,axis=1)]
slice_movies=slice_movie[:1000]

In [127]:
slice_movies['cast_score']=slice_movies.apply(lambda x: cumulated_score(actors_awards,x['cast'],int(x['startYear'])),axis=1)

C:\Users\souhail\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [128]:
slice_movies.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,cast,count,cast_score
24535,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,\N,118,"Comedy,Fantasy,Romance","[nm0000212, nm0413168, nm0000630, nm0005227, n...",72,36
50553,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,\N,122,Drama,"[nm0001379, nm0462648, nm0000953, nm0001782, n...",76,858
50671,tt0069204,movie,Sabse Bada Sukh,Sabse Bada Sukh,0,2018,\N,\N,"Comedy,Drama","[nm0037026, nm2147526, nm1027929, nm1875977, n...",13,145
62008,tt0085953,movie,Devil Fetus,Mo tai,0,2019,\N,84,Horror,"[nm0150874, nm1845142, nm0387360, nm4628925, n...",11,0
63754,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,\N,100,"Comedy,Horror,Sci-Fi","[nm0864851, nm0828288, nm0933983, nm0329491, n...",90,0


In [132]:
slice_movies.cast_score.describe()

count    1000.000000
mean       45.191000
std       105.405355
min         0.000000
25%         0.000000
50%         0.000000
75%        43.000000
max       958.000000
Name: cast_score, dtype: float64

In [ ]:
crew=pd.read_csv('data/crew.tsv',sep='\t')